In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

## Import Dataset

In [ ]:
data = pd.read_csv('../input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv')
data.describe()

In [ ]:
description=np.load('../input/book-description/Book_description.npy')
data['Description']=description
data.head()

## DATA CLEANING

In [ ]:
data.columns

## Strings

## Name

In [ ]:
print("Name:", '\n', data['Name'].value_counts())

In [ ]:
display(data[data['Name']=='Publication Manual of the American Psychological Association, 6th Edition'].head())
display(data[data['Name']=='StrengthsFinder 2.0'].head())
display(data[data['Name']=="Oh, the Places You'll Go!"].head())

# Dropping duplicates values
print("shape of the dataset before dropping duplicates is : {}".format(data.shape))
data.drop_duplicates(inplace=True)
print("shape of the dataset after dropping duplicates is : {}".format(data.shape))

## Genre

In [ ]:
print("Genre:", '\n', data['Genre'].value_counts())

sns.countplot('Genre', data=data, palette='Set3')
plt.show()

## Numbers

In [ ]:
numbers = ['User Rating', 'Reviews', 'Price']

data.loc[:,numbers].hist(color='salmon', figsize=(20,10), edgecolor='black', bins=10)
plt.show()

rating_binned = pd.cut(data['User Rating'], bins=10)
rating_binned = pd.DataFrame(pd.DataFrame(rating_binned).groupby('User Rating').size(), columns=['User Rating Count']).reset_index()

Reviews_binned = pd.cut(data['Reviews'], bins=10)
Reviews_binned = pd.DataFrame(pd.DataFrame(Reviews_binned).groupby('Reviews').size(), columns=['Reviews Count']).reset_index()

Price_binned = pd.cut(data['Price'], bins=10)
Price_binned = pd.DataFrame(pd.DataFrame(Price_binned).groupby('Price').size(), columns=['Price Count']).reset_index()

display(pd.concat([rating_binned, Reviews_binned, Price_binned], axis=1))

## Dates(year)

In [ ]:
for genre in ['Fiction','Non Fiction']:
    temp_fiction = data[data['Genre']==genre]
    sns.lmplot(x='Year',y='Price',data=temp_fiction, fit_reg=False)
    plt.title('{} Books'.format(genre))
    plt.xlabel('Reviews')
    plt.ylabel('Price')
    plt.show()

## Data Sorting

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,5))

rating = data.groupby('Year')['User Rating'].mean()
rating = rating.reset_index()

sns.lineplot(x='Year', y='User Rating', data=data, ci=None, markers=True, ax=ax[0])
ax[0].set_xticks(ticks=data['Year'].value_counts(ascending=True).index)

sns.lineplot(x='Year', y='User Rating', hue='Genre', data=data, ci=None, markers=True, ax=ax[1])
ax[1].set_xticks(ticks=data['Year'].value_counts(ascending=True).index)

plt.show()

time = pd.DataFrame(data.groupby('Year')['User Rating'].mean())
time_genre = pd.DataFrame(data.groupby(['Genre', 'Year'])['User Rating'].mean())

time['Rating Fiction'] = list(np.around(time_genre.loc['Fiction'].reset_index()['User Rating'], 3))
time['Rating Non Fiction'] = list(np.around(time_genre.loc['Non Fiction'].reset_index()['User Rating'], 3))
display(time)







## Prices behaviour

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,5))

price = data.groupby('Year')['Price'].mean()
price = price.reset_index()

sns.lineplot(x='Year', y='Price', data=data, ci=None, markers=True, ax=ax[0])
ax[0].set_xticks(ticks=data['Year'].value_counts(ascending=True).index)

sns.lineplot(x='Year', y='Price', data=data, ci=None, markers=True, ax=ax[1], hue='Genre')
ax[1].set_xticks(ticks=data['Year'].value_counts(ascending=True).index)

plt.show()

time = pd.DataFrame(data.groupby('Year')['Price'].mean())
time_genre = pd.DataFrame(data.groupby(['Genre', 'Year'])['Price'].mean())

time['Price Fiction'] = list(np.around(time_genre.loc['Fiction'].reset_index()['Price'], 3))
time['Price Non Fiction'] = list(np.around(time_genre.loc['Non Fiction'].reset_index()['Price'], 3))
display(time)


## Prices and User Rating

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))

ax.set_title('Average Price and Average User Rating')

ax.plot(data.groupby('Year')['User Rating'].mean())
ax.tick_params('y', colors='blue')
ax.set_ylabel('User Rating', color='blue')

ax2 = ax.twinx()
ax2.plot(data.groupby('Year')['Price'].mean(), color='darkorange')
ax2.tick_params('y', colors='darkorange')
ax2.set_ylabel('Price', color='darkorange')

ax.set_xticks(ticks=data['Year'].value_counts(ascending=True).index)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,5))

fiction = data[data['Genre']=='Fiction']

Nonfiction = data[data['Genre']=='Non Fiction']


ax[0].set_title('Average Price and Average User Rating (Fiction)')

ax[0].plot(fiction.groupby('Year')['User Rating'].mean())
ax[0].tick_params('y', colors='blue')
ax[0].set_ylabel('User Rating', color='blue')

ax2 = ax[0].twinx()
ax2.plot(fiction.groupby('Year')['Price'].mean(), color='darkorange')
ax2.tick_params('y', colors='darkorange')
ax2.set_ylabel('Price', color='darkorange')

ax[0].set_xticks(ticks=data['Year'].value_counts(ascending=True).index)


ax[1].set_title('Average Price and Average User Rating (Non Fiction)')

ax[1].plot(Nonfiction.groupby('Year')['User Rating'].mean())
ax[1].tick_params('y', colors='blue')
ax[1].set_ylabel('User Rating', color='blue')

ax3 = ax[1].twinx()
ax3.plot(Nonfiction.groupby('Year')['Price'].mean(), color='darkorange')
ax3.tick_params('y', colors='darkorange')
ax3.set_ylabel('Price', color='darkorange')

ax[1].set_xticks(ticks=data['Year'].value_counts(ascending=True).index)


plt.show()

## Number of reviews and rating relationship

In [ ]:
data['Reviews/Rating'] = data['Reviews']/data['User Rating']
data.sort_values('Reviews/Rating', ascending=False)

## PREDICTION/ANALYSIS USING ML TECHNIQUE

## Content Based Recommender System

In [ ]:
data['Description']=data['Description'].fillna('')


def rem_stopwords_tokenize(data,name):
      
    def getting(sen):
        example_sent = sen

        stop_words = set(stopwords.words('english')) 

        word_tokens = word_tokenize(example_sent) 

        filtered_sentence = [w for w in word_tokens if not w in stop_words] 

        filtered_sentence = [] 

        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w) 
        return filtered_sentence
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x
    
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
rem_stopwords_tokenize(data,'Description')

# Making a function to lemmatize all the words
lemmatizer = WordNetLemmatizer() 
def lemmatize_all(data,name):
    arr=data[name]
    a=[]
    for i in arr:
        b=[]
        for j in i:
            x=lemmatizer.lemmatize(j,pos='a')
            x=lemmatizer.lemmatize(x)
            b.append(x)
        a.append(b)
    data[name]=a
    
lemmatize_all(data,'Description')

def set_rec_finder(na,number):
    def intersection(lst1, lst2): 
        return list(set(lst1) & set(lst2)) 
    def find_rec(name):
        x=list(data[data['Name']==name]['Description'].values)[0]
        score=[]
        for i in range(len(data)):
            score.append([len(intersection(x,data['Description'].values[i])),data['Name'].values[i]])
        return score
    recommendations=find_rec(na)
    recommendations.sort(reverse=True)
    recommendations=np.array(recommendations)
    ans=recommendations[:number,1]
    for i in ans:
        print(i)
        

In [ ]:
set_rec_finder('The Meltdown (Diary of a Wimpy Kid Book 13)',10)